# Plot the PWV resolution requirements from the notebook LoopOnndPWVResolutionfromMagResolution_LSSTfilters.ipynb And Scale with Airmass

- author Sylvie Dagoret-Campagne
- affiliation IJCLab
- creation date : 2025/10/30
- last update : 2025/10/30
- last update : 2025/11/03 : airmass x PWV
- last update : 2025/11/04 : Implement the fit

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os,sys
import re
import pandas as pd

from astropy.io import fits
from astropy import units as u
from astropy import constants as c

plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

props = dict(boxstyle='round', facecolor='white', alpha=0.5)

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=SyntaxWarning)

In [ ]:
from scipy import interpolate
import yaml

In [ ]:
machine_name = os.uname().nodename
dm_version = "w_2025_42"
path_rubinsimphot = f"repos/repos_{dm_version}/rubinsimphot/src"
#path_rubinsimphot = "repos/repos_w_2024_17/rubinsimphot/src"
if 'sdf' in machine_name:
    #machine_name_usdf = 'sdfrome001'
    print("Set environment for USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)
elif 'dagoret-nb' in machine_name:
    print("Set environment for USDF Rubin Science Platform")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)    
elif 'mac' in machine_name:
    print("Be sure to run this notebook in conda environment named conda_py313")
else:
    print(f"Your current machine name is {machine_name}. Check your python environment")

In [ ]:
# where are stored the figures
pathfigs = "figs_PWVResolutionRequirementsScaleAirmass"
prefix = "pwv-resoreq-am"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
def Get_SED_Pickles():
    seddir = os.path.join(fdir, 'pysynphot', 'pickles')
    seddir_uvi = os.path.join(seddir,"dat_uvi")
    seddir_uvk = os.path.join(seddir,"dat_uvk")
    all_pickles_uvi = sorted(os.listdir(seddir_uvi))
    all_pickles_uvk = sorted(os.listdir(seddir_uvk))
    file_ref = os.path.join(seddir_uvk, "pickles_uk.fits")
    hdul = fits.open(file_ref)
    df_pickle = pd.DataFrame(hdul[1].data)
    NSED = len(df_pickle)

    for index in np.arange(NSED):
        filename = df_pickle.loc[index,"FILENAME"].strip()+".fits"
        fullfilename = os.path.join(seddir_uvk,filename) 
        hdul = fits.open(fullfilename)
        dff = pd.DataFrame(hdul[1].data)

In [ ]:
def GenerateMultiValues(mean,sigma,size, lognorm_flag=True):
    """
    """
    if lognorm_flag:
        mu = np.log(mean**2/np.sqrt(mean**2+sigma**2))
        sig = np.sqrt(np.log(1+sigma**2/mean**2))
        all_values = np.random.lognormal(mean=mu, sigma=sig,size=size)
    else:
        mu = mean
        sig = sigma
        all_values = np.random.normal(mu,sig,size=size)
        
    return all_values

In [ ]:
def CalculateMagsAndMagResolutions(pwv_values, pc,the_sed):
    """
    """

    # compute standard magnitude form the average called std
    mag_std = {}
    adu_std = {}
    atm_bands = pc.bandpass_total_std
    for index,f in enumerate(filter_tagnames) :
        mag_std[f] = the_sed.calc_mag(atm_bands[f])
        adu_std[f] = -2.5*np.log10(the_sed.calc_adu(atm_bands[f],photoparams))

    
    # magnitudes from non stadard pwv
    df = pd.DataFrame(columns = ["pwv","magu","magg","magr","magi","magz","magy","aduu","adug","adur","adui","aduz","aduy"])

    ## loop on pwv values
    for idx_pwv,pwv in enumerate(pwv_values):
        mag_nonstd = {}
        adu_nonstd = {}
        atm_bands = pc.coll_bandpass_total_nonstd[idx_pwv] 

        
        for index,f in enumerate(filter_tagnames) :
            mag_nonstd[f] = the_sed.calc_mag(atm_bands[f])
            adu_nonstd[f] = -2.5*np.log10(the_sed.calc_adu(atm_bands[f],photoparams))

        # add this entry
        df.loc[idx_pwv] = [pwv, mag_nonstd["u"],mag_nonstd["g"],mag_nonstd["r"],mag_nonstd["i"],mag_nonstd["z"],mag_nonstd["y"],
                       adu_nonstd["u"],adu_nonstd["g"],adu_nonstd["r"],adu_nonstd["i"],adu_nonstd["z"],adu_nonstd["y"]] 

    df = df[["pwv","aduu","adug","adur","adui","aduz","aduy"]]

    # compute the magnitude difference
    for index,f in enumerate(filter_tagnames) :
        label_in = f'adu{f}'
        label_out =f'd_adu{f}'
        df[label_out] = (df[label_in]- adu_std[f])*1000. 

    # Drop absolute mags and keep mag difference
    df = df.drop(labels=["aduu","adug","adur","adui","aduz","aduy"],axis=1)


    #compute relative color difference
    df["d_R-I"] = df["d_adur"] -  df["d_adui"]
    df["d_I-Z"] = df["d_adui"] -  df["d_aduz"]
    df["d_Z-Y"] = df["d_aduz"] -  df["d_aduy"]

    return df 

In [ ]:
def GetdPWVvsPWV_FromMagResolution(all_PWV_values,all_DPWV_values, magresocut=5.0):
    """
    """
    sel_dpwv = []
    sel_df = []
    sel_pwv = []

    # loop on PWV values
    for pwv0 in all_PWV_values:

        # initialize atmosphere for the typical average conditions pwv0
        pc = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)


        # create a flat SED and noramize it to have mag 20 in z
        the_sed_flat = Sed()
        the_sed_flat.set_flat_sed()
        the_sed_flat.name = 'flat'
        zmag = 20.0
        flux_norm = the_sed_flat.calc_flux_norm(zmag, pc.bandpass_total_std['z'])
        the_sed_flat.multiply_flux_norm(flux_norm)
    
        # loop on PWV resolution
        for dpwv in all_DPWV_values:
            if dpwv >= pwv0:
                continue
                
            # compute the subsamples with varying PWV
            pwv_samples = GenerateMultiValues(pwv0,dpwv, NSAMPLES,lognorm_flag=True)
            pwv_samples= pwv_samples[np.where(np.logical_and(pwv_samples>0., pwv_samples<20.))[0]]
           
            pc.CalculateMultiObs(am0,pwv_samples,oz0,tau0,beta0)

            #compute distribution for magnitude resolution
            df = CalculateMagsAndMagResolutions(pwv_samples, pc, the_sed_flat)
            df_stat = df.describe()
            rms_y = df_stat.loc["std"]["d_aduy"]
            rms_zy = df_stat.loc["std"]["d_Z-Y"]

            if rms_y <=  magresocut: 
                print(f"pwv0 = {pwv0:.3f} mm , dpwv = {dpwv:.3f} mm , rms_y = {rms_y:.2f} mmag rms_z-y = {rms_zy:.2f} mmag")
                sel_dpwv.append(dpwv)
                sel_df.append(df)
                sel_pwv.append(pwv0)
                break
                
    return np.array(sel_pwv), np.array(sel_dpwv), sel_df
        

In [ ]:
# reference flux in Jy
F0 = ((0.*u.ABmag).to(u.Jy)).value
F0

## Imports dedicated to this work

- import the atmospheric transparency emulator (instead of using libradtran code).
- import rubin sim
- import libPhotometricCorrections : encapsulate uninteresting calculation details

### libradtran Emulator

In [ ]:
from importlib.metadata import version
the_ver = version('getObsAtmo')
print(f"Version of getObsAtmo : {the_ver}")

In [ ]:
from getObsAtmo import ObsAtmo
emul = ObsAtmo("LSST")

In [ ]:
WL = emul.GetWL()

#### Library to fit atmosphere

In [ ]:
import sys
sys.path.append('../lib')
#import libAtmosphericFit

#### Library that encapsulate calculations for Photometric correction

In [ ]:
# This package encapsulate the calculation on calibration used in this nb
from libPhotometricCorrections import *

In [ ]:
def set_photometric_parameters(exptime, nexp, readnoise=None):
    # readnoise = None will use the default (8.8 e/pixel). Readnoise should be in electrons/pixel.
    photParams = PhotometricParameters(exptime=exptime, nexp=nexp, readnoise=readnoise)
    return photParams

In [ ]:
def scale_sed(ref_mag, ref_filter, sed):
    fluxNorm = sed.calc_flux_norm(ref_mag, lsst_std[ref_filter])
    sed.multiply_flux_norm(fluxNorm)
    return sed

In [ ]:
# set default photometric parameters to compute ADU
photoparams = set_photometric_parameters(30, 1 , readnoise=None)

### Function to fit

- $X = z \cdot PWV$

$$ dX = \frac{1}{\alpha} \frac{\ln(10)}{2.5} \frac{dm_Y}{K_Y}\cdot X^{(1-\alpha)} = \beta \cdot dm_Y \cdot X^{1-\alpha}$$

$$\frac{dX}{X} = \frac{\ln(10)}{2.5} \frac{1}{\alpha K_Y} \frac{dm_Y}{X^\alpha} = \beta \cdot \frac{dm_Y}{X^\alpha}$$

In [ ]:
def func01(x,alpha,beta):
    """
    """
    return np.power(x,1-alpha)*beta

In [ ]:
def func1(x,alpha,beta,dm):
    """
    """
    return np.power(x,1-alpha)*beta*dm

In [ ]:
def func02(x,alpha,beta):
    """
    """
    return np.power(x,-alpha)*beta

In [ ]:
def func2(x,alpha,beta,dm):
    """
    """
    return np.power(x,-alpha)*beta*dm

#### library rubin_sim defining LSST parameters, namely for photometric calculations

In [ ]:
from rubinsimphot.phot_utils import Bandpass, Sed
from rubinsimphot.data import get_data_dir

## Configuration

In [ ]:
am0 = 1.2    # airmass
am_num = int(am0*10.)
pwv0 = 4.0  # Precipitable water vapor vertical column depth in mm
oz0 = 300.  # Ozone vertical column depth in Dobson Unit (DU)
ncomp=1     # Number of aerosol components
tau0= 0.0 # Vertical Aerosol depth (VAOD) 
beta0 = 1.2 # Aerosol Angstrom exponent

### Initialisation of Atmospheric corrections

In [ ]:
pc = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)

### Check standard atmosphere

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
axs.plot(pc.WL,pc.atm_std,'k-')
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Standard atmosphere transmission")

### Check LSST instrument throughput

Photometric Correction package should find the instrumental passband of LSST

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index]) 
    axs.fill_between(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Instrument throughput (LSST)")

### Check LSST standard Filter throughputs

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index]) 
    axs.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Total filter throughput (LSST)")

## Import results

In [ ]:
datapath = "data/curves_dpwvpwv"

In [ ]:
airmass_sel = [1., 1.2, 1.5, 2.0]
airmass_sel_ls = ["-.","-","--",":"]
NAM = len(airmass_sel)

In [ ]:
all_data_sigma = []
for am in airmass_sel:
    am_num = int(am*10.)
    filename = f"pwvdpwvdata_resomagY_airmass{am_num:0}.yaml"
    fullfilename = os.path.join(datapath ,filename)
    with open(fullfilename, 'r') as infile:
        data = yaml.safe_load(infile)
        all_data_sigma.append(data)
        
all_data_delta = []
for am in airmass_sel:
    am_num = int(am*10.)
    filename = f"pwvdpwvdata_biasmagY_airmass{am_num:0}.yaml"
    fullfilename = os.path.join(datapath ,filename)
    with open(fullfilename, 'r') as infile:
        data = yaml.safe_load(infile)
        all_data_delta.append(data)
        

## Plot PWV Resolution

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), layout="constrained")

# Couleurs et labels des résolutions photométriques
color_map = {
    '10mmag': ('k', "$\\Delta m(Y) = 10$ mmag"),
    '05mmag':  ('b', "$\\Delta m(Y) = 5$ mmag"),
    '01mmag':  ('r', "$\\Delta m(Y) = 1$ mmag"),
}

# Styles de lignes et leurs labels (pour légende secondaire)
ls_labels = {
    '-.':  "$Z = 1.0$",
    '-': "$Z = 1.2$",
    '--': "$Z = 1.5$",
    ':':  "$Z = 2.0$",
}

# Boucle principale
for key, (color, label) in color_map.items():

    # loop on airmass
    for idx_am in range(NAM):
        ls = airmass_sel_ls[idx_am]
        the_data_sigma = all_data_sigma[idx_am]
        the_data_delta = all_data_delta[idx_am]
    
   
        # Exemple : key = '10mmag' → X = 'pwv10mmag', Y = 'dpwv10mmag'
        X = np.array(the_data_sigma[f'pwv{key}'])*airmass_sel[idx_am]
        Y = np.array(the_data_sigma[f'dpwv{key}'])*airmass_sel[idx_am]

        X1 = np.array(the_data_delta[f'pwv{key}'])*airmass_sel[idx_am]
        Y1 = np.array(the_data_delta[f'dpwv{key}'])*airmass_sel[idx_am]

        X = X[:-4]
        Y = Y[:-4]
        
        ax.plot(X, Y, marker="o", color=color, ls=ls, label=label)
        ax.plot(X1, Y1, marker="+", color=color, ls=":")
        
        # Ajout d’un léger remplissage autour de la courbe (±5 % par exemple)
        fill_factor = 0.05
        #fill_factor = 0.15
        ax.fill_between(X, Y * (1 - fill_factor), Y * (1 + fill_factor),
                        color=color, alpha=0.15)

# Axes et titre
ax.set_xlabel("$airmass \\cdot PWV$ (mm)", fontsize=13)
ax.set_ylabel("$airmass \\cdot \\sigma$ PWV (mm)", fontsize=13)
ax.set_title("PWV resolution vs PWV for different magnitude resolutions (Y band)", fontsize=15)
#ax.set_yscale("log")
ax.grid(True, which="both", ls="--", alpha=0.4)

# Gestion des légendes séparées
# → On crée deux légendes : une pour les couleurs, une pour les styles
handles_color = [
    plt.Line2D([], [], color=color, lw=2, label=label)
    for color, label in [v for v in color_map.values()]
]

handles_ls = [
    plt.Line2D([], [], color="gray", ls=ls, lw=2, label=label)
    for ls, label in ls_labels.items()
]

# Première légende (couleurs)
legend1 = ax.legend(handles=handles_color, loc="upper left", title="Magnitude resolution")
ax.add_artist(legend1)  # pour ne pas qu'elle soit remplacée

# Deuxième légende (styles)
ax.legend(handles=handles_ls, loc="lower right", title="Airmass")

figname =f"{pathfigs}/{prefix}_amdeltapwv_vs_amPWV_linvscale"+figtype
plt.savefig(figname)

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), layout="constrained")

# Couleurs et labels des résolutions photométriques
color_map = {
    '10mmag': ('k', "$\\Delta m(Y) = 10$ mmag"),
    '05mmag':  ('b', "$\\Delta m(Y) = 5$ mmag"),
    '01mmag':  ('r', "$\\Delta m(Y) = 1$ mmag"),
}

# Styles de lignes et leurs labels (pour légende secondaire)
ls_labels = {
    '-.':  "$Z = 1.0$",
    '-': "$Z = 1.2$",
    '--': "$Z = 1.5$",
    ':':  "$Z = 2.0$",
}

magname_to_mag = {
    '10mmag': 10.,
    '05mmag': 5.,
    '01mmag': 1., 
}

# Boucle principale
for key, (color, label) in color_map.items():

    # loop on airmass
    for idx_am in range(NAM):
        ls = airmass_sel_ls[idx_am]
        the_data_sigma = all_data_sigma[idx_am]
        the_data_delta = all_data_delta[idx_am]
    
   
        # Exemple : key = '10mmag' → X = 'pwv10mmag', Y = 'dpwv10mmag'
        X = np.array(the_data_sigma[f'pwv{key}'])*airmass_sel[idx_am]
        Y = np.array(the_data_sigma[f'dpwv{key}'])*airmass_sel[idx_am]

        X1 = np.array(the_data_delta[f'pwv{key}'])*airmass_sel[idx_am]
        Y1 = np.array(the_data_delta[f'dpwv{key}'])*airmass_sel[idx_am]

        X = X[:-4]
        Y = Y[:-4]
        
        ax.plot(X, Y, marker="o", color=color, ls=ls, label=label)
        ax.plot(X1, Y1, marker="+", color=color, ls=":")
        
        # Ajout d’un léger remplissage autour de la courbe (±5 % par exemple)
        fill_factor = 0.05
        #fill_factor = 0.15
        ax.fill_between(X, Y * (1 - fill_factor), Y * (1 + fill_factor),
                        color=color, alpha=0.15)

# Axes et titre
ax.set_xlabel("$airmass \\cdot PWV$ (mm)", fontsize=13)
ax.set_ylabel("$airmass \\cdot \\sigma$ PWV (mm)", fontsize=13)
ax.set_title("PWV resolution vs PWV for different magnitude resolutions (Y band)", fontsize=15)
ax.set_yscale("log")
ax.grid(True, which="both", ls="--", alpha=0.4)

# Gestion des légendes séparées
# → On crée deux légendes : une pour les couleurs, une pour les styles
handles_color = [
    plt.Line2D([], [], color=color, lw=2, label=label)
    for color, label in [v for v in color_map.values()]
]

handles_ls = [
    plt.Line2D([], [], color="gray", ls=ls, lw=2, label=label)
    for ls, label in ls_labels.items()
]

# Première légende (couleurs)
legend1 = ax.legend(handles=handles_color, loc="upper left", title="Magnitude resolution")
ax.add_artist(legend1)  # pour ne pas qu'elle soit remplacée

# Deuxième légende (styles)
ax.legend(handles=handles_ls, loc="lower right", title="Airmass")

figname =f"{pathfigs}/{prefix}_amdeltapwv_vs_amPWV_logvscale"+figtype
plt.savefig(figname)

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), layout="constrained")

# Couleurs et labels des résolutions photométriques
color_map = {
    '10mmag': ('k', "$\\Delta m(Y) = 10$ mmag"),
    '05mmag':  ('b', "$\\Delta m(Y) = 5$ mmag"),
    '01mmag':  ('r', "$\\Delta m(Y) = 1$ mmag"),
}

# Styles de lignes et leurs labels (pour légende secondaire)
ls_labels = {
    '-.':  "$Z = 1.0$",
    '-': "$Z = 1.2$",
    '--': "$Z = 1.5$",
    ':':  "$Z = 2.0$",
}

magname_to_mag = {
    '10mmag': 10.,
    '05mmag': 5.,
    '01mmag': 1., 
}

# Loop on magnitude data
for key, (color, label) in color_map.items():

    # Boucle principale sur l'airmass
    for idx_am in range(NAM):
        ls = airmass_sel_ls[idx_am]
        the_data_sigma = all_data_sigma[idx_am]
        the_data_delta = all_data_delta[idx_am]
 
        # Exemple : key = '10mmag' → X = 'pwv10mmag', Y = 'dpwv10mmag'
        X = np.array(the_data_sigma[f'pwv{key}'])*airmass_sel[idx_am]
        Y = np.array(the_data_sigma[f'dpwv{key}'])*airmass_sel[idx_am]

        X1 = np.array(the_data_delta[f'pwv{key}'])*airmass_sel[idx_am]
        Y1 = np.array(the_data_delta[f'dpwv{key}'])*airmass_sel[idx_am]

        X = X[:-4]
        Y = Y[:-4]

        
        Y = Y/magname_to_mag[key]
        Y1 = Y1/magname_to_mag[key]
    
        
        ax.plot(X, Y, marker="o", color=color, ls=ls, label=label)
        ax.plot(X1, Y1, marker="+", color=color, ls=":")
        
        # Ajout d’un léger remplissage autour de la courbe (±5 % par exemple)
        fill_factor = 0.05
        #fill_factor = 0.15
        ax.fill_between(X, Y * (1 - fill_factor), Y * (1 + fill_factor),
                        color=color, alpha=0.15)

# Axes et titre
ax.set_xlabel("$airmass \\cdot PWV$ (mm)", fontsize=13)
ax.set_ylabel("$airmass \\cdot \\sigma(PWV1/mmag) /(\\sigma(m)/1mmag)$", fontsize=13)
ax.set_title("PWV resolution vs PWV for different magnitude resolutions (Y band)", fontsize=15)
#ax.set_yscale("log")
ax.grid(True, which="both", ls="--", alpha=0.4)

# Gestion des légendes séparées
# → On crée deux légendes : une pour les couleurs, une pour les styles
handles_color = [
    plt.Line2D([], [], color=color, lw=2, label=label)
    for color, label in [v for v in color_map.values()]
]

handles_ls = [
    plt.Line2D([], [], color="gray", ls=ls, lw=2, label=label)
    for ls, label in ls_labels.items()
]

# Première légende (couleurs)
legend1 = ax.legend(handles=handles_color, loc="upper left", title="Magnitude resolution")
ax.add_artist(legend1)  # pour ne pas qu'elle soit remplacée

# Deuxième légende (styles)
ax.legend(handles=handles_ls, loc="lower right", title="Airmass")

figname =f"{pathfigs}/{prefix}_amdeltapwv_vs_amPWV_linvscale-divdeltam"+figtype
plt.savefig(figname)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), layout="constrained")

# Couleurs et labels des résolutions photométriques
color_map = {
    '10mmag': ('k', "$\\Delta m(Y) = 10$ mmag"),
    '05mmag':  ('b', "$\\Delta m(Y) = 5$ mmag"),
    '01mmag':  ('r', "$\\Delta m(Y) = 1$ mmag"),
}

# Styles de lignes et leurs labels (pour légende secondaire)
ls_labels = {
    '-.':  "$Z = 1.0$",
    '-': "$Z = 1.2$",
    '--': "$Z = 1.5$",
    ':':  "$Z = 2.0$",
}

magname_to_mag = {
    '10mmag': 10.,
    '05mmag': 5.,
    '01mmag': 1., 
}

# Boucle principale
for key, (color, label) in color_map.items():
    
    for idx_am in range(NAM):
        ls = airmass_sel_ls[idx_am]
        the_data_sigma = all_data_sigma[idx_am]
        the_data_delta = all_data_delta[idx_am]
    
        # Exemple : key = '10mmag' → X = 'pwv10mmag', Y = 'dpwv10mmag'
        X = np.array(the_data_sigma[f'pwv{key}'])*airmass_sel[idx_am]
        Y = np.array(the_data_sigma[f'dpwv{key}'])*airmass_sel[idx_am]

        X1 = np.array(the_data_delta[f'pwv{key}'])*airmass_sel[idx_am]
        Y1 = np.array(the_data_delta[f'dpwv{key}'])*airmass_sel[idx_am]

        X = X[:-4]
        Y = Y[:-4]

        Y = Y/magname_to_mag[key]
        Y1 = Y1/magname_to_mag[key]
            
        
        ax.plot(X, Y, marker="o", color=color, ls=ls, label=label)
        ax.plot(X1, Y1, marker="+", color=color, ls=":")
        
        # Ajout d’un léger remplissage autour de la courbe (±5 % par exemple)
        fill_factor = 0.05
        #fill_factor = 0.15
        ax.fill_between(X, Y * (1 - fill_factor), Y * (1 + fill_factor),
                        color=color, alpha=0.15)

# Axes et titre
ax.set_xlabel("$airmass \\times PWV$ (mm)", fontsize=13)
ax.set_ylabel("$airmass \\times \\sigma PWV (mm) \Delta m $ ", fontsize=13)
ax.set_title("PWV resolution vs PWV for different magnitude resolutions (Y band)", fontsize=15)
ax.set_yscale("log")
ax.grid(True, which="both", ls="--", alpha=0.4)

# Gestion des légendes séparées
# → On crée deux légendes : une pour les couleurs, une pour les styles
handles_color = [
    plt.Line2D([], [], color=color, lw=2, label=label)
    for color, label in [v for v in color_map.values()]
]

handles_ls = [
    plt.Line2D([], [], color="gray", ls=ls, lw=2, label=label)
    for ls, label in ls_labels.items()
]

# Première légende (couleurs)
legend1 = ax.legend(handles=handles_color, loc="upper left", title="Magnitude resolution")
ax.add_artist(legend1)  # pour ne pas qu'elle soit remplacée

# Deuxième légende (styles)
ax.legend(handles=handles_ls, loc="lower right", title="Airmass")

figname =f"{pathfigs}/{prefix}_amdeltapwv_vs_amPWV_logvscale-divdeltam"+figtype
plt.savefig(figname)

plt.show()


## Fit

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), layout="constrained")

# Couleurs et labels des résolutions photométriques
color_map = {
    '10mmag': ('k', "$\\Delta m(Y) = 10$ mmag"),
    '05mmag':  ('b', "$\\Delta m(Y) = 5$ mmag"),
    '01mmag':  ('r', "$\\Delta m(Y) = 1$ mmag"),
}

magname_to_mag = {
    '10mmag': 10.,
    '05mmag': 5.,
    '01mmag': 1., 
}

# Styles de lignes et leurs labels (pour légende secondaire)
ls_labels = {
    '-.':  "$Z = 1.0$",
    '-': "$Z = 1.2$",
    '--': "$Z = 1.5$",
    ':':  "$Z = 2.0$",
}

# Dictionnaire pour stocker les résultats de fit
fit_results = {}

# Boucle principale sur les magnitudes
for key, (color, label) in color_map.items():

    dm = magname_to_mag[key]

    XX = []
    YY = []

    # loop on airmass
    for idx_am in range(NAM):
        
        ls = airmass_sel_ls[idx_am]
        the_data_sigma = all_data_sigma[idx_am]
        the_data_delta = all_data_delta[idx_am]
    
   
        # Exemple : key = '10mmag' → X = 'pwv10mmag', Y = 'dpwv10mmag'
        X = np.array(the_data_sigma[f'pwv{key}'])*airmass_sel[idx_am]
        Y = np.array(the_data_sigma[f'dpwv{key}'])*airmass_sel[idx_am]

       

        X = X[:-4]
        Y = Y[:-4]
        
        ax.plot(X, Y, marker="o", color=color,ls=ls ,label=label)

        XX.append(X)
        YY.append(Y)
       
    XX= np.hstack(XX)    
    YY = np.hstack(YY)
    xfit = np.linspace(XX.min(),XX.max(),100)

    # fix the dm parameter
    def func_to_fit(x,alpha,beta):
        return func1(x,alpha,beta,dm=dm)
    alpha0 = 0.5
    beta0 = 0.1

    popt, pcov = curve_fit(func_to_fit, XX, YY, p0=[alpha0, beta0])
    yfit = func_to_fit(xfit,*popt)
    ax.plot(xfit,yfit,"-",lw=3,color=color)

    # Stocker les résultats du fit pour la légende
    fit_results[key] = {
        "alpha": popt[0],
        "beta": popt[1],
        "color": color,
        "label": label
    }
    

# Axes et titre
ax.set_xlabel("$airmass \\cdot PWV$ (mm)", fontsize=13)
ax.set_ylabel("$airmass \\cdot \\sigma$ PWV (mm)", fontsize=13)
ax.set_title("PWV resolution vs PWV for different magnitude resolutions (Y band)", fontsize=15)
#ax.set_yscale("log")
ax.grid(True, which="both", ls="--", alpha=0.4)



# Légendes
handles_color = [
    plt.Line2D([], [], color=color, lw=2, label=f"{label}\n$\\alpha={fit_results[k]['alpha']:.3f}$, $\\beta={fit_results[k]['beta']:.3f}$")
    for k, (color, label) in color_map.items()
]

handles_ls = [
    plt.Line2D([], [], color="gray", ls=ls, lw=2, label=label)
    for ls, label in ls_labels.items()
]

# Première légende : magnitudes + paramètres
legend1 = ax.legend(handles=handles_color, loc="upper left", title="Magnitude resolution & fit parameters")
ax.add_artist(legend1)

# Deuxième légende : airmass
ax.legend(handles=handles_ls, loc="lower right", title="Airmass")




figname =f"{pathfigs}/{prefix}_fit_amdeltapwv_vs_amPWV_linvscale"+figtype
plt.savefig(figname)

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), layout="constrained")

# Couleurs et labels des résolutions photométriques
color_map = {
    '10mmag': ('k', "$\\Delta m(Y) = 10$ mmag"),
    '05mmag':  ('b', "$\\Delta m(Y) = 5$ mmag"),
    '01mmag':  ('r', "$\\Delta m(Y) = 1$ mmag"),
}

# Styles de lignes et leurs labels (pour légende secondaire)
ls_labels = {
    '-.':  "$Z = 1.0$",
    '-': "$Z = 1.2$",
    '--': "$Z = 1.5$",
    ':':  "$Z = 2.0$",
}

magname_to_mag = {
    '10mmag': 10.,
    '05mmag': 5.,
    '01mmag': 1., 
}

# Loop on magnitude data
for key, (color, label) in color_map.items():

    dm = magname_to_mag[key]
    XX = []
    YY = []

    # Boucle principale sur l'airmass
    for idx_am in range(NAM):
        ls = airmass_sel_ls[idx_am]
        the_data_sigma = all_data_sigma[idx_am]
      
 
        # Exemple : key = '10mmag' → X = 'pwv10mmag', Y = 'dpwv10mmag'
        X = np.array(the_data_sigma[f'pwv{key}'])*airmass_sel[idx_am]
        Y = np.array(the_data_sigma[f'dpwv{key}'])*airmass_sel[idx_am]


        X = X[:-4]
        Y = Y[:-4]
  
        Y = Y/magname_to_mag[key]
      
        
        ax.plot(X, Y, marker="o", color=color, ls=ls, label=label)

        XX.append(X)
        YY.append(Y)
       
XX= np.hstack(XX)    
YY = np.hstack(YY)
xfit = np.linspace(XX.min(),XX.max(),100)


def func_to_fit(x,alpha,beta):
    return func01(x,alpha,beta)
alpha0 = 0.5
beta0 = 0.1

popt, pcov = curve_fit(func_to_fit, XX, YY, p0=[alpha0, beta0])
yfit = func_to_fit(xfit,*popt)
ax.plot(xfit,yfit,"-",lw=3,color="green")

# Erreurs sur les paramètres
perr = np.sqrt(np.diag(pcov))

# Texte à afficher
fit_text = (
    "fit results : \n"
    f"$\\alpha = {popt[0]:.3f} \\pm {perr[0]:.3f}$\n"
    f"$\\beta = {popt[1]:.3f} \\pm {perr[1]:.3f}$"
)

# Ajouter le texte sur le graphique
ax.text(
    0.4, 0.98, fit_text,
    transform=ax.transAxes,
    fontsize=12,
    verticalalignment='top',color="g",
    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8)
)

        
# Axes et titre
ax.set_xlabel("$airmass \\cdot PWV$ (mm)", fontsize=13)
ax.set_ylabel("$airmass \\cdot \\sigma(PWV1/mmag) /(\\sigma(m)/1mmag)$", fontsize=13)
ax.set_title("PWV resolution vs PWV for different magnitude resolutions (Y band)", fontsize=15)
#ax.set_yscale("log")
ax.grid(True, which="both", ls="--", alpha=0.4)

# Gestion des légendes séparées
# → On crée deux légendes : une pour les couleurs, une pour les styles
handles_color = [
    plt.Line2D([], [], color=color, lw=2, label=label)
    for color, label in [v for v in color_map.values()]
]

handles_ls = [
    plt.Line2D([], [], color="gray", ls=ls, lw=2, label=label)
    for ls, label in ls_labels.items()
]

# Première légende (couleurs)
legend1 = ax.legend(handles=handles_color, loc="upper left", title="Magnitude resolution")
ax.add_artist(legend1)  # pour ne pas qu'elle soit remplacée

# Deuxième légende (styles)
ax.legend(handles=handles_ls, loc="lower right", title="Airmass")

figname =f"{pathfigs}/{prefix}_fit_amdeltapwv_vs_amPWV_linvscale-divdeltam"+figtype
plt.savefig(figname)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), layout="constrained")

# Couleurs et labels des résolutions photométriques
color_map = {
    '10mmag': ('k', "$\\Delta m(Y) = 10$ mmag"),
    '05mmag':  ('b', "$\\Delta m(Y) = 5$ mmag"),
    '01mmag':  ('r', "$\\Delta m(Y) = 1$ mmag"),
}

# Styles de lignes et leurs labels (pour légende secondaire)
ls_labels = {
    '-.':  "$Z = 1.0$",
    '-': "$Z = 1.2$",
    '--': "$Z = 1.5$",
    ':':  "$Z = 2.0$",
}

magname_to_mag = {
    '10mmag': 10.,
    '05mmag': 5.,
    '01mmag': 1., 
}

# Loop on magnitude data
for key, (color, label) in color_map.items():

    dm = magname_to_mag[key]
    XX = []
    YY = []

    # Boucle principale sur l'airmass
    for idx_am in range(NAM):
        ls = airmass_sel_ls[idx_am]
        the_data_sigma = all_data_sigma[idx_am]
      
 
        # Exemple : key = '10mmag' → X = 'pwv10mmag', Y = 'dpwv10mmag'
        X = np.array(the_data_sigma[f'pwv{key}'])*airmass_sel[idx_am]
        Y = np.array(the_data_sigma[f'dpwv{key}'])*airmass_sel[idx_am]


        X = X[:-4]
        Y = Y[:-4]
  
        Y = Y/magname_to_mag[key]
      
        
        ax.plot(X, Y, marker="o", color=color, ls=ls, label=label)

        XX.append(X)
        YY.append(Y)
       
XX= np.hstack(XX)    
YY = np.hstack(YY)
xfit = np.linspace(XX.min(),XX.max(),100)


def func_to_fit(x,alpha,beta):
    return func01(x,alpha,beta)
alpha0 = 0.5
beta0 = 0.1

popt, pcov = curve_fit(func_to_fit, XX, YY, p0=[alpha0, beta0])
yfit = func_to_fit(xfit,*popt)
ax.plot(xfit,yfit,"-",lw=3,color="green")

# Erreurs sur les paramètres
perr = np.sqrt(np.diag(pcov))

# Texte à afficher
fit_text = (
    "fit results : \n"
    f"$\\alpha = {popt[0]:.3f} \\pm {perr[0]:.3f}$\n"
    f"$\\beta = {popt[1]:.3f} \\pm {perr[1]:.3f}$"
)

# Ajouter le texte sur le graphique
ax.text(
    0.4, 0.98, fit_text,
    transform=ax.transAxes,
    fontsize=12,
    verticalalignment='top',color="g",
    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8)
)

        
# Axes et titre
ax.set_xlabel("$airmass \\cdot PWV$ (mm)", fontsize=13)
ax.set_ylabel("$airmass \\cdot \\sigma(PWV1/mmag) /(\\sigma(m)/1mmag)$", fontsize=13)
ax.set_title("PWV resolution vs PWV for different magnitude resolutions (Y band)", fontsize=15)
#ax.set_yscale("log")
ax.grid(True, which="both", ls="--", alpha=0.4)

# Gestion des légendes séparées
# → On crée deux légendes : une pour les couleurs, une pour les styles
handles_color = [
    plt.Line2D([], [], color=color, lw=2, label=label)
    for color, label in [v for v in color_map.values()]
]

handles_ls = [
    plt.Line2D([], [], color="gray", ls=ls, lw=2, label=label)
    for ls, label in ls_labels.items()
]

# Première légende (couleurs)
legend1 = ax.legend(handles=handles_color, loc="upper left", title="Magnitude resolution")
ax.add_artist(legend1)  # pour ne pas qu'elle soit remplacée

# Deuxième légende (styles)
ax.legend(handles=handles_ls, loc="lower right", title="Airmass")

ax.set_yscale("log")
ax.set_ylim(0.01,3.)

figname =f"{pathfigs}/{prefix}_fit_amdeltapwv_vs_amPWV_logvscale-divdeltam"+figtype
plt.savefig(figname)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), layout="constrained")

# Couleurs et labels des résolutions photométriques
color_map = {
    '10mmag': ('k', "$\\Delta m(Y) = 10$ mmag"),
    '05mmag':  ('b', "$\\Delta m(Y) = 5$ mmag"),
    '01mmag':  ('r', "$\\Delta m(Y) = 1$ mmag"),
}

magname_to_mag = {
    '10mmag': 10.,
    '05mmag': 5.,
    '01mmag': 1., 
}

# Styles de lignes et leurs labels (pour légende secondaire)
ls_labels = {
    '-.':  "$Z = 1.0$",
    '-': "$Z = 1.2$",
    '--': "$Z = 1.5$",
    ':':  "$Z = 2.0$",
}

# Dictionnaire pour stocker les résultats de fit
fit_results = {}

# Boucle principale sur les magnitudes
for key, (color, label) in color_map.items():

    dm = magname_to_mag[key]

    XX = []
    YY = []

    # loop on airmass
    for idx_am in range(NAM):
        
        ls = airmass_sel_ls[idx_am]
        the_data_sigma = all_data_sigma[idx_am]
        the_data_delta = all_data_delta[idx_am]
    
   
        # Exemple : key = '10mmag' → X = 'pwv10mmag', Y = 'dpwv10mmag'
        X = np.array(the_data_sigma[f'pwv{key}'])*airmass_sel[idx_am]
        Y = np.array(the_data_sigma[f'dpwv{key}'])*airmass_sel[idx_am]

       

        X = X[:-4]
        Y = Y[:-4]

        Y = Y/X
        
        ax.plot(X, Y, marker="o", color=color,ls=ls ,label=label)

        XX.append(X)
        YY.append(Y)
       
    XX= np.hstack(XX)    
    YY = np.hstack(YY)
    xfit = np.linspace(XX.min(),XX.max(),100)

    # fix the dm parameter
    def func_to_fit(x,alpha,beta):
        return func2(x,alpha,beta,dm=dm)
    alpha0 = 0.5
    beta0 = 0.1

    popt, pcov = curve_fit(func_to_fit, XX, YY, p0=[alpha0, beta0])
    yfit = func_to_fit(xfit,*popt)
    ax.plot(xfit,yfit,"-",lw=3,color=color)

    # Stocker les résultats du fit pour la légende
    fit_results[key] = {
        "alpha": popt[0],
        "beta": popt[1],
        "color": color,
        "label": label
    }
    

# Axes et titre
ax.set_xlabel("$airmass \\cdot PWV$ (mm)", fontsize=13)
ax.set_ylabel("$\\frac{\\sigma PWV}{PWV}$", fontsize=13)
ax.set_title("PWV resolution vs PWV for different magnitude resolutions (Y band)", fontsize=15)
#ax.set_yscale("log")
ax.grid(True, which="both", ls="--", alpha=0.4)



# Légendes
handles_color = [
    plt.Line2D([], [], color=color, lw=2, label=f"{label}\n$\\alpha={fit_results[k]['alpha']:.3f}$, $\\beta={fit_results[k]['beta']:.3f}$")
    for k, (color, label) in color_map.items()
]

handles_ls = [
    plt.Line2D([], [], color="gray", ls=ls, lw=2, label=label)
    for ls, label in ls_labels.items()
]

# Première légende : magnitudes + paramètres
legend1 = ax.legend(handles=handles_color, loc="upper left", title="Magnitude resolution & fit parameters")
ax.add_artist(legend1)

# Deuxième légende : airmass
ax.legend(handles=handles_ls, loc="lower right", title="Airmass")




figname =f"{pathfigs}/{prefix}_fit_amdeltapwvdivpwv_vs_amPWV_linvscale"+figtype
plt.savefig(figname)

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), layout="constrained")

# Couleurs et labels des résolutions photométriques
color_map = {
    '10mmag': ('k', "$\\Delta m(Y) = 10$ mmag"),
    '05mmag':  ('b', "$\\Delta m(Y) = 5$ mmag"),
    '01mmag':  ('r', "$\\Delta m(Y) = 1$ mmag"),
}

# Styles de lignes et leurs labels (pour légende secondaire)
ls_labels = {
    '-.':  "$Z = 1.0$",
    '-': "$Z = 1.2$",
    '--': "$Z = 1.5$",
    ':':  "$Z = 2.0$",
}

magname_to_mag = {
    '10mmag': 10.,
    '05mmag': 5.,
    '01mmag': 1., 
}

# Loop on magnitude data
for key, (color, label) in color_map.items():

    dm = magname_to_mag[key]
    XX = []
    YY = []

    # Boucle principale sur l'airmass
    for idx_am in range(NAM):
        ls = airmass_sel_ls[idx_am]
        the_data_sigma = all_data_sigma[idx_am]
      
 
        # Exemple : key = '10mmag' → X = 'pwv10mmag', Y = 'dpwv10mmag'
        X = np.array(the_data_sigma[f'pwv{key}'])*airmass_sel[idx_am]
        Y = np.array(the_data_sigma[f'dpwv{key}'])*airmass_sel[idx_am]


        X = X[:-4]
        Y = Y[:-4]

        Y = Y/X
  
        Y = Y/magname_to_mag[key]
      
        
        ax.plot(X, Y, marker="o", color=color, ls=ls, label=label)

        XX.append(X)
        YY.append(Y)
       
XX= np.hstack(XX)    
YY = np.hstack(YY)
xfit = np.linspace(XX.min(),XX.max(),100)


def func_to_fit(x,alpha,beta):
    return func02(x,alpha,beta)
alpha0 = 0.5
beta0 = 0.1

popt, pcov = curve_fit(func_to_fit, XX, YY, p0=[alpha0, beta0])
yfit = func_to_fit(xfit,*popt)
ax.plot(xfit,yfit,"-",lw=3,color="green")

# Erreurs sur les paramètres
perr = np.sqrt(np.diag(pcov))

# Texte à afficher
fit_text = (
    "fit results : \n"
    f"$\\alpha = {popt[0]:.3f} \\pm {perr[0]:.3f}$\n"
    f"$\\beta = {popt[1]:.3f} \\pm {perr[1]:.3f}$"
)

# Ajouter le texte sur le graphique
ax.text(
    0.4, 0.98, fit_text,
    transform=ax.transAxes,
    fontsize=12,
    verticalalignment='top',color="g",
    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8)
)

        
# Axes et titre
ax.set_xlabel("$airmass \\cdot PWV$ (mm)", fontsize=13)
ax.set_ylabel("$\\frac{\\sigma(PWV)}{PWV} /(\\sigma(m)/1mmag)$", fontsize=13)
ax.set_title("PWV resolution vs PWV for different magnitude resolutions (Y band)", fontsize=15)
#ax.set_yscale("log")
ax.grid(True, which="both", ls="--", alpha=0.4)

# Gestion des légendes séparées
# → On crée deux légendes : une pour les couleurs, une pour les styles
handles_color = [
    plt.Line2D([], [], color=color, lw=2, label=label)
    for color, label in [v for v in color_map.values()]
]

handles_ls = [
    plt.Line2D([], [], color="gray", ls=ls, lw=2, label=label)
    for ls, label in ls_labels.items()
]

# Première légende (couleurs)
legend1 = ax.legend(handles=handles_color, loc="upper left", title="Magnitude resolution")
ax.add_artist(legend1)  # pour ne pas qu'elle soit remplacée

# Deuxième légende (styles)
ax.legend(handles=handles_ls, loc="lower right", title="Airmass")

figname =f"{pathfigs}/{prefix}_fit_deltapwvdivpwv_vs_amPWV_linvscale-divdeltam"+figtype
plt.savefig(figname)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), layout="constrained")

# Couleurs et labels des résolutions photométriques
color_map = {
    '10mmag': ('k', "$\\Delta m(Y) = 10$ mmag"),
    '05mmag':  ('b', "$\\Delta m(Y) = 5$ mmag"),
    '01mmag':  ('r', "$\\Delta m(Y) = 1$ mmag"),
}

# Styles de lignes et leurs labels (pour légende secondaire)
ls_labels = {
    '-.':  "$Z = 1.0$",
    '-': "$Z = 1.2$",
    '--': "$Z = 1.5$",
    ':':  "$Z = 2.0$",
}

magname_to_mag = {
    '10mmag': 10.,
    '05mmag': 5.,
    '01mmag': 1., 
}

# Loop on magnitude data
for key, (color, label) in color_map.items():

    dm = magname_to_mag[key]
    XX = []
    YY = []

    # Boucle principale sur l'airmass
    for idx_am in range(NAM):
        ls = airmass_sel_ls[idx_am]
        the_data_sigma = all_data_sigma[idx_am]
      
 
        # Exemple : key = '10mmag' → X = 'pwv10mmag', Y = 'dpwv10mmag'
        X = np.array(the_data_sigma[f'pwv{key}'])*airmass_sel[idx_am]
        Y = np.array(the_data_sigma[f'dpwv{key}'])*airmass_sel[idx_am]


        X = X[:-4]
        Y = Y[:-4]

        Y = Y/X
  
        Y = Y/magname_to_mag[key]
      
        
        ax.plot(X, Y, marker="o", color=color, ls=ls, label=label)

        XX.append(X)
        YY.append(Y)
       
XX= np.hstack(XX)    
YY = np.hstack(YY)
xfit = np.linspace(XX.min(),XX.max(),100)


def func_to_fit(x,alpha,beta):
    return func02(x,alpha,beta)
alpha0 = 0.5
beta0 = 0.1

popt, pcov = curve_fit(func_to_fit, XX, YY, p0=[alpha0, beta0])
yfit = func_to_fit(xfit,*popt)
ax.plot(xfit,yfit,"-",lw=3,color="green")

# Erreurs sur les paramètres
perr = np.sqrt(np.diag(pcov))

# Texte à afficher
fit_text = (
    "fit results : \n"
    f"$\\alpha = {popt[0]:.3f} \\pm {perr[0]:.3f}$\n"
    f"$\\beta = {popt[1]:.3f} \\pm {perr[1]:.3f}$"
)

# Ajouter le texte sur le graphique
ax.text(
    0.4, 0.98, fit_text,
    transform=ax.transAxes,
    fontsize=12,
    verticalalignment='top',color="g",
    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8)
)

        
# Axes et titre
ax.set_xlabel("$airmass \\cdot PWV$ (mm)", fontsize=13)
ax.set_ylabel("$\\frac{\\sigma(PWV)}{PWV} /(\\sigma(m)/1mmag)$", fontsize=13)
ax.set_title("PWV resolution vs PWV for different magnitude resolutions (Y band)", fontsize=15)
#ax.set_yscale("log")
ax.grid(True, which="both", ls="--", alpha=0.4)

# Gestion des légendes séparées
# → On crée deux légendes : une pour les couleurs, une pour les styles
handles_color = [
    plt.Line2D([], [], color=color, lw=2, label=label)
    for color, label in [v for v in color_map.values()]
]

handles_ls = [
    plt.Line2D([], [], color="gray", ls=ls, lw=2, label=label)
    for ls, label in ls_labels.items()
]

# Première légende (couleurs)
legend1 = ax.legend(handles=handles_color, loc="upper left", title="Magnitude resolution")
ax.add_artist(legend1)  # pour ne pas qu'elle soit remplacée

# Deuxième légende (styles)
ax.legend(handles=handles_ls, loc="lower right", title="Airmass")
ax.set_yscale("log")


figname =f"{pathfigs}/{prefix}_fit_deltapwvdivpwv_vs_amPWV_logvscale-divdeltam"+figtype



plt.savefig(figname)

plt.show()